## Image Embeddings

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

from os import listdir, makedirs, path
from random import shuffle

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from PIL import Image as PImage, ImageOps as PImageOps

from parameters.arquigrafia import IMAGES_PATH
from parameters.embeddings import EMBEDDINGS_PATH

from models.embedding_models import Clip, EfficientNet, ResNet, Vit

makedirs(EMBEDDINGS_PATH, exist_ok=True)

### Run embeddings

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

for idx, io_file in list(enumerate(input_files))[:]:
  input_file_path = path.join(IMAGES_PATH, io_file)
  output_file_path = path.join(EMBEDDINGS_PATH, io_file.replace(".jpg", ".json"))

  if path.isfile(output_file_path):
    continue

  if idx % 100 == 0:
    print(idx, IMAGES_PATH, io_file)

  image = PImageOps.exif_transpose(PImage.open(input_file_path).convert("RGB"))

  raw_embs = {}

  raw_embs["clip"] = Clip.get_embedding(image).tolist()
  raw_embs["efficient"] = EfficientNet.get_embedding(image).tolist()
  raw_embs["resnet"] = ResNet.get_embedding(image).tolist()
  raw_embs["vit"] = Vit.get_embedding(image).tolist()

  image_embs = {"raw": raw_embs}

  with open(output_file_path, "w", encoding="utf-8") as of:
    json.dump(image_embs, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

### Reduce Dims

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

raw_embs = {}

for idx, io_file in enumerate(input_files):
  output_file_path = path.join(EMBEDDINGS_PATH, io_file.replace(".jpg", ".json"))

  if not path.isfile(output_file_path):
    continue

  if idx % 100 == 0:
    print(idx, IMAGES_PATH, io_file)

  with open(output_file_path, "r", encoding="utf8") as f:
    image_embs = json.load(f)

  for k,e in image_embs["raw"].items():
    if k not in raw_embs:
      raw_embs[k] = []
    raw_embs[k].append(e)

In [ ]:
def cluster_pca(emb_raw, n_clusters=6, n_components=4):
  mStandard = StandardScaler()
  mPCA = PCA(n_components=n_components)
  # mCluster = KMeans(n_clusters=n_clusters)
  mCluster = GaussianMixture(n_components=n_clusters)

  emb_std = mStandard.fit_transform(emb_raw)
  emb_pca = mPCA.fit_transform(emb_std)
  emb_clusters = mCluster.fit_predict(emb_std)

  return emb_std, emb_pca, emb_clusters, 0 #mCluster.inertia_

In [ ]:
def plot_cluster_pca(clusters, pcas, title=""):
  for i in range(3):
    for j in range(i+1,3):
      plt.scatter(pcas[:,i], pcas[:,j], c=clusters, marker='o', linestyle='', alpha=0.5)
      plt.title(title)
      plt.show()

  # 3D
  fig = plt.figure(figsize=(8, 8))
  ax = fig.add_subplot(projection='3d')
  ax.scatter(pcas[:,0], pcas[:,1], pcas[:,2], c=clusters, marker='o', linestyle='', alpha=0.25)
  ax.set_title(title)
  plt.show()

In [ ]:
clip_std, clip_pca, clip_clusters, clip_wss = cluster_pca(raw_embs["clip"])
plot_cluster_pca(clip_clusters, clip_pca, title="Clip")

In [ ]:
efficient_std, efficient_pca, efficient_clusters, efficient_wss = cluster_pca(raw_embs["efficient"])
plot_cluster_pca(efficient_clusters, efficient_pca, title="Efficient Net")

In [ ]:
resnet_std, resnet_pca, resnet_clusters, resnet_wss = cluster_pca(raw_embs["resnet"])
plot_cluster_pca(resnet_clusters, resnet_pca, title="ResNet")

In [ ]:
vit_std, vit_pca, vit_clusters, vit_wss = cluster_pca(raw_embs["vit"])
plot_cluster_pca(vit_clusters, vit_pca, title="Vit")

In [15]:
emb_name = "vit"
wss = []
sil = []
for nc in range(2,8):
  _std, _, _clusters, _wss = cluster_pca(raw_embs[emb_name], n_clusters=nc)
  _sil = silhouette_score(_std, _clusters)
  wss.append(_wss)
  sil.append(_sil)

In [ ]:
plt.plot(range(2,8), wss)
plt.show()

plt.plot(range(2,8), sil)
plt.show()

In [ ]:
emb_name = "resnet"
wss = []
sil = []
for nc in range(2,8):
  _std, _, _clusters, _wss = cluster_pca(raw_embs[emb_name], n_clusters=nc)
  _sil = silhouette_score(_std, _clusters)
  wss.append(_wss)
  sil.append(_sil)

In [ ]:
plt.plot(range(2,8), wss)
plt.show()

plt.plot(range(2,8), sil)
plt.show()

In [ ]:
input_files = sorted([f for f in listdir(IMAGES_PATH) if f.endswith("jpg")])

In [ ]:
m_clusters = resnet_clusters

for c in np.unique(m_clusters):
  this_cluster = np.where(m_clusters == c)[0]
  shuffle(this_cluster)
  fig, axes = plt.subplots(nrows=8, ncols=8)
  fig.set_size_inches(10, 10)
  fig.set_dpi(72)

  fig.suptitle(f"Vit Cluster {c}")
  for ciidx, ax in enumerate(axes.flat):
    iidx = this_cluster[ciidx]
    img = PImage.open(path.join(IMAGES_PATH, input_files[iidx])).convert("RGB")
    img = PImageOps.exif_transpose(img).resize((128,128))
    ax.imshow(img)
    ax.axis("off")

  plt.tight_layout()
  plt.show()

In [ ]:
# TODO: PCA
# TODO: write output

  # with open(output_file_path, "w", encoding="utf-8") as of:
  #   json.dump(image_embs, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

## Test Embeddings

In [ ]:
imgs = [
  PImage.open(IMAGES_PATH+"/100.jpg"),
  PImage.open(IMAGES_PATH+"/101.jpg"),
  PImage.open(IMAGES_PATH+"/1000.jpg"),
  PImage.open(IMAGES_PATH+"/1001.jpg"),
  PImage.open(IMAGES_PATH+"/1010.jpg"),
  PImage.open(IMAGES_PATH+"/1011.jpg"),
]

img = imgs[0]

for i in imgs:
  display(i.resize((128,128)))

In [ ]:
emb = ResNet.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = ResNet.get_embedding(img)
print(emb.shape)

In [ ]:
emb = EfficientNet.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = EfficientNet.get_embedding(img)
print(emb.shape)

In [ ]:
emb = Vit.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = Vit.get_embedding(img)
print(emb.shape)

In [ ]:
emb = Clip.get_embedding(imgs)
print(emb.shape)

emb_diff = (emb - emb[1]).pow(2).sum(dim=1).pow(0.5)
emb_diff.argsort()

In [ ]:
emb = Clip.get_embedding(img)
print(emb.shape)